# Spotted Python Week 5 Tutorial
In this script, we aim to practice the following skills:
- Scaling Relations
- Fitting a linear curve
- BPT diagrams

In [ ]:
# import libraries
from math import log10, floor
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colours
from astropy.io import ascii
from astropy.table import Table, join
from scipy.odr import *

## Scaling Relations
One of the scaling relation explored by the SAMI survey is, $$S_{0.5}=\sqrt{0.5V_{rot}^2+\sigma^2},$$ where $V_{rot}$ is the rotational velocity, and $\sigma$ is the velocity dispersion of the galaxy.

### **Example 1:**
Import the 4 csv files relevant to option 2 of the computational essay using ascii. Then use the rotational velocity and velocity dispersion to generate a log stellar mass versus $S_{0.5}$ plot for the GAMA objects.

In [ ]:
# import data
morphology = ascii.read("samiDR3VisualMorphology.csv", format='basic', delimiter=',', guess=False)
kinematics = ascii.read("samiDR3StelKin.csv", format='basic', delimiter=',', guess=False)
clusters = ascii.read("samiDR3InputCatClusters.csv", format='basic', delimiter=',', guess=False)
gama = ascii.read("InputCatGAMADR3.csv", format='basic', delimiter=',', guess=False)

In [ ]:
# for reference, let's print these tables
morphology

In [ ]:
kinematics

In [ ]:
clusters

In [ ]:
gama

If we look at the tables, it can be seen that some of the kinematics data associated with the gama and clusters objects are listed in the separate kinematics csv file. Therefore we must first combined the data sets by matching the objects names.

It is also important to note that the velocity dispersion is given under the column *sigma_re*, the flux weighted $V/\sigma$ within effective radius effective ellipse is given under the column *vsigma_re*, and the log of the stellar mass is given under *mstar*.

In [ ]:
# Combine the gama table with the kinematics table, by cross matching the list of CATIDs
gama_kinematics = join(gama, kinematics, keys='catid')

# Calculate S0.5 for gama galaxies
gama_s05 = np.sqrt(0.5*(gama_kinematics['vsigma_re']*gama_kinematics['sigma_re'])**2 +
                   gama_kinematics['sigma_re']**2)
gama_s05_no_nan = gama_s05[~np.isnan(gama_s05)] # to remove nan

# log of stellar mass
log_gama_mass = gama_kinematics['mstar']
log_gama_mass_no_nan = log_gama_mass[~np.isnan(gama_s05)] # to delete corresponding rows

# Plot the log-log relation between stellar mass and S0.5
plt.plot(np.log(gama_s05_no_nan), log_gama_mass_no_nan, '+',
         markersize=5, color = 'royalblue', label='GAMA')
plt.xlabel('$logS_{0.5}$ $[km/s]$') # x-axis label
plt.ylabel('$logM_*$ $[M_\odot]$') # y-axis label
plt.legend(loc='lower right') # show legend

### **Excercise 1:**
Repeat example 1 with the cluster objects.

## Linear Fitting
As we know, linear fitting is an important skill in physics, and is a key method to determine the relationship between two parameters.
### **Example 2:**
Using the GAMA objects, fit a linear function to relate $S_{0.5}$ and stellar mass.

In [ ]:
# define a general linear function
def linear(parameters, x):
    m, c = parameters
    return m*x+c

In [ ]:
# Fitting the linear curve to the data
model = Model(linear) # making the defined function into a model
# takes the real data and fits it to the model with an initial guess for the parameters
gama_odr = ODR(RealData(np.log(gama_s05_no_nan), log_gama_mass_no_nan),
                 model, beta0=[2,7])
gama_output = gama_odr.run()
gama_output.pprint()

In [ ]:
# plot the original figure with the line of best fit
plt.plot(np.log(gama_s05_no_nan), log_gama_mass_no_nan, '+',
         markersize=5, color = 'royalblue', label='GAMA') # orginal data
plt.plot(np.log(gama_s05_no_nan), linear(gama_output.beta, np.log(gama_s05_no_nan)),
         color='red', label = "y = ({:.2f} +/- {:.2f}) * x\n+ ({:.2f} +/- {:.2f})"
         .format(gama_output.beta[0], gama_output.sd_beta[0],gama_output.beta[1], gama_output.sd_beta[1]))

plt.xlabel('$logS_{0.5}$ $[km/s]$') # x-axis label
plt.ylabel('$logM_*$ $[M_\odot]$') # y-axis label
plt.legend(loc='lower right') # show legend

### **Excercise 2:**
Find the line of best fit for the galaxies in clusters. Then compare the trends from the gama galaxies and galaxies in clusters

## BPT Diagram
### **Example and Exercise 3:**
Using the classifications given in Kewley et al. (2016), plot the emission line galaxy on a $\log⁡([OIII]/[OII])$ versus $\log⁡([OI]/H_\alpha)$ plot:
1. HII & Composites: $\log⁡([OIII]/[OII])<−1.701 \log⁡([OI]/H_\alpha)−2.163$
2. LINERS: $\log⁡([OIII]/[OII])<1.0 \log⁡([OI]/H_\alpha)+0.7$
3. Seyferts: $\log⁡([OIII]/[OII])>1.0 \log⁡([OI]/H_\alpha)+0.7$


In [ ]:
# Finding the ratios for a chosen object
# SDSS ID: 1237648720693756163
# import data from shared csv file
emission_lines = ascii.read('sdss_spectra.csv', format='basic', delimiter=',')

# define lines
oi = emission_lines['[O I]']
oii = emission_lines['[O II]1'] + emission_lines['[O II]2']
oiii = emission_lines['[O III]']
halpha = emission_lines['[Ha]']

# find ratios
oi_ha = np.log(oi/halpha)
oiii_oii = np.log(oiii/oii)

In [ ]:
# Plot the diagnostic lines:
# let y = log([OIII]/[OII]) and x = log([OI]/H_alpha)

# find intersection of the diagnostic lines
intersect = (-2.163-0.7)/2.701

# defining the lines
x_hii= np.linspace(-3,1,1000)
y_hii = -1.701*x_hii-2.163
x_liner = np.linspace(intersect,1,1000)
y_liner = x_liner+0.7

# plotting diagnostic lines
plt.plot(x_hii, y_hii, 'black')
plt.plot(x_liner,y_liner, 'black')

# plot the emission galaxy
plt.plot(oi_ha, oiii_oii, '*', color='darkviolet')

# plot display settings
plt.xlabel('$\log([OI]/H$\\alpha$)$')
plt.ylabel('$\log([OIII]/[OII])$')
plt.margins(x=0)
plt.margins(y=0)

# Labels
plt.text(-2.1, -2.5, 'HII', fontsize=12, color = 'crimson', fontweight='bold')
plt.text(-1.3, 1.4, 'Seyferts', fontsize = 12, color = 'mediumblue', fontweight='bold')
plt.text(0, -0.7, 'LINERs', fontsize = 12, color = 'mediumseagreen', fontweight='bold')